# HOMEWORK 3

In [1]:
def install(package):
    try:
        import subprocess
        import sys
        subprocess.call([sys.executable, "-m", "pip", "install", package])
        print("Library installed")
    except:
        print("Cannot install the library")

In [2]:
try:
    from langdetect import detect
except:
    print("The library LanfDetect is not in the repository, I will install it")
    install('langdetect')
    from langdetect import detect

try:
    from texttable import Texttable
except:
    print("The library TextTable is not in the repository, I will install it")
    install('texttable')
    from texttable import Texttable

try:
    import geopy as g
except:
    print("The library TextTable is not in the repository, I will install it")
    install('geopy')
    import geopy as g



In [3]:
import pandas as pd
import unicodecsv as csv # We also import the unicodecsv library to handle any Unicode objects in the CSV report...as csv.
import nltk
import string
import pickle

from nltk import word_tokenize
from nltk.corpus import stopwords
from langdetect import detect

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/fabiomontello/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

STEP 1 - DATA

In [33]:
DIR = 'data/docs/' # C:/Users/david/Desktop/DATA SCIENCE/AMD/documents/

df = pd.read_csv("data/Airbnb_Texas_Rentals.csv", sep=",") # we read the dataset

In [34]:
df = df.drop('Unnamed: 0',axis = 1) #this column is useless for our analisys
df = df[pd.notnull((df['description'])) & pd.notnull((df['latitude'])) & pd.notnull((df['longitude'])) & (df['description'] != "\\" ) & (df['description'] != "." )] #If the row has no description, for us is useless, so we drop it
df = df.replace(r'\\n',' ', regex=True) #Remove all the new lines in the dataframe
df['language'] = 'nd'
df.head()

,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url,language
0,$27,2,Humble,May 2016,Welcome to stay in private room with queen bed...,30.020138,-95.293996,2 Private rooms/bathroom 10min from IAH airport,https://www.airbnb.com/rooms/18520444?location...,nd
1,$149,4,San Antonio,November 2010,"Stylish, fully remodeled home in upscale NW – ...",29.503068,-98.447688,Unique Location! Alamo Heights - Designer Insp...,https://www.airbnb.com/rooms/17481455?location...,nd
2,$59,1,Houston,January 2017,'River house on island close to the city' A w...,29.829352,-95.081549,River house near the city,https://www.airbnb.com/rooms/16926307?location...,nd
3,$60,1,Bryan,February 2016,Private bedroom in a cute little home situated...,30.637304,-96.337846,Private Room Close to Campus,https://www.airbnb.com/rooms/11839729?location...,nd
4,$75,2,Fort Worth,February 2017,Welcome to our original 1920's home. We recent...,32.747097,-97.286434,The Porch,https://www.airbnb.com/rooms/17325114?location...,nd


STEP 2 - WRITE DOCUMENTS

In [ ]:
for row in df.iterrows():
    file_name = DIR + 'doc_'+str(row[0])+'.tsv'
    with open(file_name,'wb') as file: # the wb means we are writing a file in binary mode.
        csv_writer = csv.writer(file, delimiter = '\t') #now we are writing the files as tab separated files.
        csv_writer.writerow(list(row[1].values)) #writerow want a list object ad input, so we used list.
        file.close() #Once the file is written, we close it, so we free some memory


In [8]:
docdict = {}

languages = {'en': 'english', 'de': 'german', 'es': 'spanish', 'nl': 'dutch'}

for i, row in df[:].iterrows(): #We limit to 5 just to work quickly

    #Remove punctuation
    text = str(row['description']).translate({key: None for key in string.punctuation}) + ' '+ str(row['title']).translate({key: None for key in string.punctuation})
    
    if(text):
        try:
            lan = detect(str(text))
            if (lan in languages):
                languages[lan]
        
                #The text here is Spanish, we can decide what to do
                df.loc[i,'language'] = lan
        
                #Remove all the stopwords and all the non alphabetic words
                tostem = ([x.lower() for x in text.split(' ') if x.lower() not in stopwords.words(languages[lan]) and x.isalpha()])
        
                #Stem all the words
                docdict[i] = [nltk.stem.SnowballStemmer(languages[lan]).stem(word) for word in tostem]
            else:
                pass
        except:
            pass

# languages
docdict

{0: ['welcom',
  'stay',
  'privat',
  'room',
  'queen',
  'bed',
  'detach',
  'privat',
  'bathroom',
  'second',
  'anoth',
  'privat',
  'bedroom',
  'sofa',
  'bed',
  'avail',
  'addit',
  'addit',
  'iah',
  'airport',
  'airport',
  'avail',
  'privat',
  'iah',
  'airport'],
 1: ['fulli',
  'remodel',
  'home',
  'upscal',
  'nw',
  'alamo',
  'height',
  'amaz',
  'locat',
  'hous',
  'conveni',
  'locat',
  'quiet',
  'beauti',
  'season',
  'prestigi',
  'neighborhood',
  'close',
  'loop',
  'featur',
  'open',
  'floor',
  'origin',
  'hardwood',
  'full',
  'bathroom',
  'independ',
  'room',
  'sleep',
  'european',
  'inspir',
  'kitchen',
  'driveway',
  'park',
  'uniqu',
  'alamo',
  'height',
  'design',
  'inspir'],
 2: ['hous',
  'island',
  'close',
  'well',
  'maintain',
  'river',
  'hous',
  'san',
  'jacinto',
  'river',
  'extra',
  'room',
  'temporari',
  'river',
  'hous',
  'near',
  'citi'],
 3: ['privat',
  'bedroom',
  'cute',
  'littl',
  'home',


In [9]:
voc = {}
cont = 0
for key, value in docdict.items():
    for elem in value:
        if(elem not in voc):
            voc[elem] = cont
            cont += 1
print(voc)

{'welcom': 0, 'stay': 1, 'privat': 2, 'room': 3, 'queen': 4, 'bed': 5, 'detach': 6, 'bathroom': 7, 'second': 8, 'anoth': 9, 'bedroom': 10, 'sofa': 11, 'avail': 12, 'addit': 13, 'iah': 14, 'airport': 15, 'fulli': 16, 'remodel': 17, 'home': 18, 'upscal': 19, 'nw': 20, 'alamo': 21, 'height': 22, 'amaz': 23, 'locat': 24, 'hous': 25, 'conveni': 26, 'quiet': 27, 'beauti': 28, 'season': 29, 'prestigi': 30, 'neighborhood': 31, 'close': 32, 'loop': 33, 'featur': 34, 'open': 35, 'floor': 36, 'origin': 37, 'hardwood': 38, 'full': 39, 'independ': 40, 'sleep': 41, 'european': 42, 'inspir': 43, 'kitchen': 44, 'driveway': 45, 'park': 46, 'uniqu': 47, 'design': 48, 'island': 49, 'well': 50, 'maintain': 51, 'river': 52, 'san': 53, 'jacinto': 54, 'extra': 55, 'temporari': 56, 'near': 57, 'citi': 58, 'cute': 59, 'littl': 60, 'situat': 61, 'covet': 62, 'garden': 63, 'acr': 64, 'access': 65, 'campus': 66, 'recent': 67, 'purchas': 68, 'mile': 69, 'outsid': 70, 'downtown': 71, 'fort': 72, 'worth': 73, 'happi

In [10]:
invidx = {} # 1: [0,2,5], 245: [1, 4, 7] ->  1: [(0, 1), (2, 3), 5], 245: [1, 4, 7]

for key, value in docdict.items():
    #
    for elem in value:
        if(voc[elem] in invidx):
            invidx[voc[elem]] = invidx[voc[elem]] + [key]
        else:
            invidx[voc[elem]] = [key]
print(invidx)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [11]:
# def jaccard_sim(list1, list2):
#     a = list1
#     b = list2
#     c = a.intersection(b)
#     return float(len(c))/ len(a) + len(b) - len(c)
def searchJac(string, n):
    languages = {'en': 'english', 'de': 'german', 'es': 'spanish', 'nl': 'dutch'}
    text = string
    lan = detect(str(text))
    if(lan not in languages):
        lan = 'en'
    tostem = ([x.lower() for x in text.split(' ') if x.lower() not in stopwords.words(languages[lan]) and x.isalpha()])
    result = [nltk.stem.SnowballStemmer(languages[lan]).stem(word) for word in tostem]
    
    listofdocs = {}
    for elem in result:
        if(elem in voc):
            if(len(listofdocs) == 0):
                listofdocs = set(invidx[voc[elem]])
            else:
                listofdocs = listofdocs & set(invidx[voc[elem]])
    t = Texttable()
    t.add_row(['Title', 'Description', 'City', 'URL'])
    for elem in listofdocs:
        file_name = DIR + 'doc_'+str(elem)+'.tsv'
        with open(file_name,'rb') as file: # the wb means we are writing a file in binary mode.
            text = str(file.read()).split('\\t')
            t.add_row([text[7], text[4], text[2], text[8]])
            file.close()

    print(t.draw())

In [12]:
searchJac('Increíble habitación lugares', 10)

+---------------------+---------------------+-------------+--------------------+
| Title               | Description         | City        | URL                |
+---------------------+---------------------+-------------+--------------------+
| Cuarto luminoso     | Lugares de          | Irving      | https://www.airbnb |
|                     | inter\xc3\xa9s:     |             | .com/rooms/1607339 |
|                     | transporte          |             | 3?location=Coppell |
|                     | p\xc3\xbablico,     |             | %2C%20TX\r\n'      |
|                     | parques. Te va a    |             |                    |
|                     | encantar mi espacio |             |                    |
|                     | porque Hay mucho    |             |                    |
|                     | verde y espacios    |             |                    |
|                     | con juegos para     |             |                    |
|                     | ni\x

In [ ]:
# df.loc[df['language'] == 'es']

In [13]:
#function tfidf
def tf_idf(word,doc,D,N):
    import math
    f = doc.count(word) / len(doc)
    tfidf = math.log(D/N)*f
    
    return (tfidf)

#function cosine similarity

def cosine_sim(x , y):
    import numpy as np
    
    a = dot(x,y)
    
    b = np.linalg.norm(x)
    c = np.linalg.norm(y)
    
    cos = (a/(b*c))
    
    return (cos)
    

    

In [14]:
# Here I have to create the dictionary with the term ID as key and then the documents where the term id is and also the tfidf.

invidx2 = invidx.copy()
alldocs = []
for key, value in invidx2.items():
    alldocs = alldocs + invidx2[key]
    invidx2[key] = set(value)

Ndocs = len(set(alldocs))
print(Ndocs)

18154


In [15]:
inv_voc = {v: k for k, v in voc.items()}
invidx3 = invidx2.copy()

for key, value in invidx2.items():
    tmp = []
    for elem in value:
        file_name = DIR + 'doc_'+str(elem)+'.tsv'
        with open(file_name,'rb') as file: # the wb means we are writing a file in binary mode.
            text = str(file.read()).split('\\t')
            text = text[4].split(' ')
            file.close()
        x = tf_idf(inv_voc[key], text, Ndocs, len(value))
        tmp.append((elem,x))
    invidx3[key] = tmp

print(invidx3)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [ ]:
print(invidx3[1])

In [16]:

# with open(DIR+'dicIndex.json', 'w') as file:
#      file.write(json.dumps(invidx3)) # use `json.loads` to do the reverse
        
with open(DIR+'dicIndex.pkl', 'wb') as file:
        pickle.dump(invidx3, file, pickle.HIGHEST_PROTOCOL)

In [25]:
def searchCos(string, n):
    with open(DIR+'dicIndex.pkl', 'rb') as file:
        data = pickle.load(file)
    languages = {'en': 'english', 'de': 'german', 'es': 'spanish', 'nl': 'dutch'}
    text = string
    lan = detect(str(text))
    if(lan not in languages):
        lan = 'en'
    tostem = ([x.lower() for x in text.split(' ') if x.lower() not in stopwords.words(languages[lan]) and x.isalpha()])
    result = [nltk.stem.SnowballStemmer(languages[lan]).stem(word) for word in tostem]
    
    listofdocs = {}
    for elem in result:
        if(elem in voc):
            if(len(listofdocs) == 0):
                listofdocs = set(invidx3[voc[elem]])
            else:
                listofdocs = listofdocs & set(invidx3[voc[elem]])
    listofdocs = sorted(list(listofdocs), key=lambda x: -x[1])
    print(listofdocs)
    t = Texttable()
    t.add_row(['Title', 'Description', 'City', 'URL', 'Similarity'])
    for elem in enumerate(listofdocs):
        file_name = DIR + 'doc_'+str(elem[0])+'.tsv'
        with open(file_name,'rb') as file: # the wb means we are writing a file in binary mode.
            text = str(file.read()).split('\\t')
            t.add_row([text[7], text[4], text[2], text[8], elem[1][1]])
            file.close()

    print(t.draw())

In [26]:
searchCos('Increíble habitación lugares', 10)

[(3837, 0.1669768069137191), (113, 0.10145426242858883), (11388, 0.10145426242858883), (10959, 0.0), (5675, 0.0), (121, 0.0)]
+-----------------+----------------+-------------+----------------+------------+
| Title           | Description    | City        | URL            | Similarity |
+-----------------+----------------+-------------+----------------+------------+
| 2 Private       | Welcome to     | Humble      | https://www.ai | 0.167      |
| rooms/bathroom  | stay in        |             | rbnb.com/rooms |            |
| 10min from IAH  | private room   |             | /18520444?loca |            |
| airport         | with queen bed |             | tion=Cleveland |            |
|                 | and detached   |             | %2C%20TX\r\n'  |            |
|                 | private        |             |                |            |
|                 | bathroom on    |             |                |            |
|                 | the second     |             |              

In [35]:
def findcenter(lat, long):
    return [sum(lat)/len(lat), sum(long)/len(long)]

In [36]:
df.groupby
latvect = df.groupby('city')['latitude'].apply(list)
lonvect = df.groupby('city')['longitude'].apply(list)

In [44]:
centers = {}
for key, elem in latvect.iteritems():
    centers[key] = findcenter(elem, lonvect[key])
print(centers)


{'Abilene': [32.431428385508895, -99.73726780556402], 'Addison': [32.95160598682212, -96.8401761074865], 'Alamo Heights': [29.474794396944297, -98.4633664006353], 'Alba': [32.85701867700555, -95.63068630422336], 'Aledo': [32.70364574496464, -97.6136277849499], 'Allen': [33.10127488942106, -96.6514103337796], 'Alpine': [30.295878116942017, -103.6495421233394], 'Alvarado': [32.3646032317485, -97.2372120865714], 'Alvin': [29.369816367414423, -95.25332148363741], 'Amarillo': [35.18169062193556, -101.87203441056539], 'Anahuac': [29.716429672581466, -94.68871257824055], 'Angleton': [29.184437119368802, -95.42516713485179], 'Anna': [33.321053236466696, -96.58525312812691], 'Anson': [32.7083641907311, -99.8787513760017], 'Aransas Pass': [27.90444246733221, -97.15904543147897], 'Argyle': [33.107980244573284, -97.10813892640556], 'Arlington': [32.727846835162076, -97.11198993777695], 'Art': [30.7196880078644, -99.00696703114184], 'Atascocita': [29.97066913509663, -95.18705017414457], 'Athens': [

In [45]:
latvect

city
Abilene              [32.436160808846395, 32.4378100178221, 32.4593...
Addison              [32.9465723482768, 32.940307161415895, 32.9627...
Alamo Heights                 [29.474794396944297, 29.474794396944297]
Alba                              [32.8689589568922, 32.8450783971189]
Aledo                [32.7010606798284, 32.7010606798284, 32.701060...
Allen                [33.0953349702649, 33.0913116724907, 33.103466...
Alpine               [30.3593094068398, 30.3586718311464, 30.358719...
Alvarado                                            [32.3646032317485]
Alvin                [29.408111714979, 29.370482167393103, 29.33114...
Amarillo             [35.1321310223634, 35.220309839763, 35.2014079...
Anahuac              [29.7609703284601, 29.627609415913895, 29.7607...
Angleton                          [29.1968067085344, 29.1720675302032]
Anna                          [33.321053236466696, 33.321053236466696]
Anson                                               [32.7083641907311]
A

In [46]:
lonvect

city
Abilene              [-99.7740097883848, -99.69983992222859, -99.73...
Addison              [-96.845189026702, -96.850780349526, -96.82859...
Alamo Heights                   [-98.4633664006353, -98.4633664006353]
Alba                          [-95.62744324427129, -95.63392936417542]
Aledo                [-97.6697994513307, -97.6697994513307, -97.669...
Allen                [-96.70009475036949, -96.6306320971205, -96.68...
Alpine               [-103.66127370762501, -103.662563887663, -103....
Alvarado                                           [-97.2372120865714]
Alvin                [-95.16089885230801, -95.2756294907732, -95.29...
Amarillo             [-101.90844758092199, -101.889328846992, -101....
Anahuac              [-94.6801981662681, -94.7057649765489, -94.680...
Angleton                        [-95.4197222153541, -95.4306120543495]
Anna                          [-96.58525312812691, -96.58525312812691]
Anson                                              [-99.8787513760017]
A